In [1]:
import pandas as pd
import sqlalchemy as sa
from math import ceil
import os

### data extract helper function

In [2]:
def get_engine(svr,db):
    """given server and db, return sqlalchemy engine"""

    constr = 'mssql+pyodbc://{}/{}?'.format(svr,db)
    constr += 'driver=SQL+Server+Native+Client+11.0?'
    constr += 'trusted_connection=yes'
    
    return sa.create_engine(constr)

### get main data set

In [3]:
# get sqlalchemy engine
engine = get_engine('EUSETL','ETLDB')

In [4]:
df = pd.read_sql_table('ZenGAProject',con=engine)

In [5]:
df.dtypes

DBKey                        int64
BUS_DAT             datetime64[ns]
DOC_ID                       int64
PMT_SEQ_NO                   int64
CustomerKey                  int64
VisitNumber                  int64
VisitCount                   int64
TicketDate          datetime64[ns]
TicketTime                  object
PriorVisits                  int64
SaleLines                    int64
ReturnLines                  int64
GiftCardLines                int64
ProduceLines                 int64
MeatLines                    int64
FishLines                    int64
OilLines                     int64
FreshPastaLines              int64
SAFOLines                    int64
RotisserieLines              int64
NetAmount                  float64
NetRetailAmount            float64
NetQSRAmount               float64
DiscountAmount             float64
StationGroup                object
UniqueItems                  int64
UniqueCategories             int64
ReturnedBags                  bool
TopItemLines        

In [7]:
df.drop(['DBKey','BUS_DAT','DOC_ID','PMT_SEQ_NO','VisitCount'], axis=1, inplace=True)

In [8]:
df.shape

(1621854, 26)

In [13]:
df.sample(5).T

,1048384,1143469,491521,25886,789839
CustomerKey,378661,803279,634479,282574,855947
VisitNumber,2,3,2,1,3
TicketDate,2014-12-04 00:00:00,2014-12-21 00:00:00,2014-02-25 00:00:00,2015-10-28 00:00:00,2014-08-07 00:00:00
TicketTime,12:23:53,17:19:13,20:17:01,17:48:15,12:34:23
PriorVisits,1,2,1,0,2
SaleLines,1,1,5,1,1
ReturnLines,0,0,0,0,0
GiftCardLines,0,0,0,0,0
ProduceLines,0,0,0,0,0
MeatLines,0,0,0,0,0


In [14]:
# export data into smaller files for github

outfp = os.path.join('data','maindata')
step = 50000
f_prefix = 'data_'

# remove old data files
for f in os.listdir(outfp):
    os.remove(os.path.join(outfp,f))

# write data to files in batches
for r in range(int(ceil(len(df) / step))):
    r_low = r*step
    r_high = (r+1)*step
    f_name = f_prefix + str(r).zfill(3) + '.csv'

    filename = os.path.join(outfp, f_name)
    
    df.iloc[r_low:r_high].to_csv(filename, index=None)

### extract ticket counts by hour

In [39]:
# construct engine
engine = get_engine('EUSETL','CPSQL')

In [40]:
# get sql query from text
with open('sql/tkt_count.sql') as fobj:
    sqltxt = fobj.read()

In [41]:
dftktct = pd.read_sql_query(sql=sqltxt, con=engine, index_col=None)

In [42]:
dftktct.head()

,TicketDate,Hr,TicketCount
0,2014-01-30,10,48
1,2015-07-21,16,357
2,2015-04-26,10,131
3,2014-04-26,16,459
4,2015-03-02,21,104


In [93]:
dftktct.to_csv('data/tkt_count.csv', index=None)